Media Cloud: Measuring Attention
================================

At this point you should be ready to query Media Cloud for data. You can use boolean query syntax - [read our query guide](https://mediacloud.org/support/query-guide) for more details about the exact syntax (it runs a [SOLR search](https://lucene.apache.org/solr/guide/6_6/the-standard-query-parser.html#the-standard-query-parser) under the hood). **This notebook demonstrates how to quickly measure attention paid to an issue by the media**.

Studying media attention is critical for understanding how much readers are exposed to an issue, and has a long traditon. Media Cloud supports investigatin attention within individual sources we track, or within collecitons or sources. We have wide global coverage with both national-level and regional/state-level collections for most countries. You can [browse our geographic collections](https://sources.mediacloud.org/#/collections/country-and-state) to see more.

The primary API method to support studying attention is exposed in our Python API as `storyCount`. You can read [the low level documentation](https://github.com/berkmancenter/mediacloud/blob/master/doc/api_2_0_spec/api_2_0_spec.md#apiv2stories_publiccount) for more details about the parameters it supports.

In [ ]:
# Grab your API key from the environment variable and create a client for talking to Media Cloud
import os, mediacloud.api
from dotenv import load_dotenv
from IPython.display import JSON
load_dotenv()  # load config from .env file
mc = mediacloud.api.MediaCloud(os.getenv('MC_API_KEY'))
mediacloud.__version__

## Attention from a Single Media Source
You can start by looking at attention from a single media source to a topic you are interested in. We have almost a million media sources in our system, but only about 100,000 of them are ones that we regularly collect stories from, via RSS feeds or more recently from their sitemaps. You can get the internal id number for any source by searching for it in our [Source Manager tool](https://sources.mediacloud.org/) and noting the ID number just under the large title on that page.

In [ ]:
# check how many stories mention "climate change" in the Washington Post (media id #2)
my_query = '"climate change" and media_id:2'
mc.storyCount(my_query)

In [ ]:
# limit this to just 2019
# note: the publish_day clause *must* be in the second argument (because of the way we have set up our DB)
# note: in solr syntax, square-brackets mean inclusive for a range, while curly braces mean exclusive
mc.storyCount(my_query, 'publish_day:[2019-01-01T00:00:00Z TO 2020-01-01T00:00:00Z}')

In [ ]:
# but that date syntax is kind of ugly, so we have a helper to produce it for you from python dates
import datetime
start_date = datetime.date(2019,1,1)
end_date = datetime.date(2020,1,1)
date_range_2019 = mc.publish_date_query(start_date, end_date) # default is start inclusive, end exclusive
mc.storyCount(my_query, date_range_2019)

In [ ]:
# you can see this over time by using the `split` argument
# this defaults to results by day, but you can pass a split_period in (day, week, month, or year)
results = mc.storyCount(my_query, date_range_2019, split=True, split_period='month')
JSON(results)

### Normalizing within a Source

Looking at absolute attention at the story level is intriguing, but you probably want to normalize this in some way to support comparisons between sources. To do this, we typically compare attention to the total number of stories we have from a source within that same timespan.

In [ ]:
relevant_stories = mc.storyCount(my_query, date_range_2019)
total_stories = mc.storyCount('media_id:2', date_range_2019)
source_ratio = relevant_stories['count'] / total_stories['count']
'{:.2%} of 2019 Washington Post stories are about "climate change"'.format(source_ratio)

## Research Within a Country - using collections

[We have wide global coverage](https://sources.mediacloud.org/#/collections/country-and-state), with sources published in a country grouped into collections. For many of these countries we also have collections of media sources published in the various states and provinces. Lets compare the source-level attention to country-level attention.

In [ ]:
# check in our collection of country-level US National media sources
us_query = '"climate change" and tags_id_media:34412234'
mc.storyCount(india_query, date_range_2019)

In [ ]:
# lets normalize this attention in the same way as we did previsouly
relevant_stories = mc.storyCount(india_query, date_range_2019)
total_stories = mc.storyCount('tags_id_media:34412234', date_range_2019)
us_country_ratio = relevant_stories['count'] / total_stories['count']
'{:.2%} of stories from national-level US media sources in 2019 mentioned "climate change"'.format(us_country_ratio)

In [ ]:
# now we can compare this to the source-level coverage
coverage_ratio = source_ratio / us_country_ratio
'"climate change" received {:.2} times as much coverage in WashPo than you might expect based on other US national papers'.format(coverage_ratio)

In [ ]:
# or compare to another country (India in this case)
relevant_stories = mc.storyCount('"climate change" and tags_id_media:34412118', date_range_2019)
total_stories = mc.storyCount('tags_id_media:34412118', date_range_2019)
india_country_ratio = relevant_stories['count'] / total_stories['count']
'{:.2%} of stories from national-level Indian media sources in 2019 mentioned "climate change"'.format(india_country_ratio)

In [ ]:
coverage_ratio =  india_country_ratio / us_country_ratio
'at the national level "climate change" is covered {:.2}x more in India than the US'.format(coverage_ratio)